In [1]:
import io
import sys
from pathlib import Path
from typing import List, Optional

from google.auth.exceptions import MutualTLSChannelError
from google.oauth2 import service_account
from googleapiclient.discovery import Resource, build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

try:
    from .. import CREDENTIAL_FILE_PATH, logger
except:
    from stock import CREDENTIAL_FILE_PATH, logger


# If modifying these scopes, delete the file token.json.
SCOPES = [
    # "https://www.googleapis.com/auth/drive.resource",
    "https://www.googleapis.com/auth/drive.metadata.readonly",
]
# Collected data is stored in Google Drive of `MyDrive/${ROOT_FOLDER_NAME}/`.
ROOT_FOLDER_NAME = "stock"


def get_root_folder_id(service: Resource) -> Optional[str]:
    """ """
    base_kwargs = {
        q: "mimeType = 'application/vnd.google-apps.folder' and name = '{ROOT_FOLDER_NAME}' and sharedWithMe",
        fileds: "nextPageToken, files(id, parents)",
        pageSize: 10,
    }
    folder = service.files().list(**base_kwargs).execute()
    while True:
        items = folder.get("files", [])
        for item in items:
            if len(item["parents"]) == 1:
                return item["id"]

        nextPageToken = folder.get("nextPageToken", "")
        if nextPageToken == "":
            break

        folder = service.files().list(**base_kwargs, pageToken=nextPageToken).execute()

    logger.info(f"Root folder not found in Google Drive. name = {ROOT_FOLDER_NAME}")
    return None


def get_path(service: Resource, file_id: str) -> Optional[Path]:
    """ """
    file = service.files().get(fileId=file_id, fileds="id, name, parents").execute()
    filepath = Path(file.get("name"))
    parent = file.get("parents")
    while parent:
        file = service.files().get(fileId=file_id, fileds="id, name, parents").execute()
        file_id = file.get("id")
        filepath = Path(file.get("name")) / filepath
        parent = file.get("parents")
    return filepath


def delete_all(service: Resource):
    """ """
    results = service.files().list(pageSize=10, fields="nextPageToken, files(id, name)").execute()

    while True:
        items = results.get("files", [])
        for item in items:
            service.files().delete(fileId=item["id"]).execute()
            print(f"{item['name']} ({item['id']}) deleted")

        nextPageToken = results.get("nextPageToken", "")
        if nextPageToken == "":
            break

        results = (
            service.files()
            .list(pageSize=10, fields="nextPageToken", pageToken=nextPageToken)
            .execute()
        )


def download_file(service: Resource, file_id: str, filepath: Path):
    try:
        file_id = file_id

        request = service.files().get_media(fileId=file_id)
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}.")

    except HttpError as error:
        print(f"An error occurred: {error}")
        file = None


def download_all(service: Resource):
    """Download all files from Google Drive to `${PROJECT_ROOT}/data`"""

    results = service.files().list(pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get("files", [])

    while True:
        items = results.get("files", [])
        for item in items:
            print(f"{item['name']} ({item['id']})")

        nextPageToken = results.get("nextPageToken", "")
        if nextPageToken == "":
            break

        results = (
            service.files()
            .list(pageSize=10, fields="nextPageToken", pageToken=nextPageToken)
            .execute()
        )


def upload_all(service: Resource):
    """Upload all files from `${PROJECT_ROOT}/data` to Google Drive"""


def get_service(
    cred_file_path: Path = CREDENTIAL_FILE_PATH, scopes: List[str] = SCOPES
) -> Optional[Resource]:
    if not cred_file_path.exists():
        logger.error(f"Credential file not found: {cred_file_path}")
        raise FileNotFoundError(f"{cred_file_path} not found")

    creds = service_account.Credentials.from_service_account_file(
        str(cred_file_path), scopes=scopes
    )

    try:
        service = build("drive", "v3", credentials=creds)
        return service
    except MutualTLSChannelError as error:
        logger.error(f"An error occurred: {error}")
    return None


[2022-08-28 11:44:22 INFO /home/kitamura/work/stock/stock/__init__.py at line 46] Set new `StreamHandler` instance to the logger.
2022-08-28 11:44:22,587 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-28 11:44:22,588 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("companies")
2022-08-28 11:44:22,589 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-28 11:44:22,590 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("companies")
2022-08-28 11:44:22,590 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-28 11:44:22,591 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("stock_time_series")
2022-08-28 11:44:22,592 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-28 11:44:22,593 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("stock_time_series")
2022-08-28 11:44:22,593 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-28 11:44:22,594 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("statistics")
2022-08-28 11:44:22,595 INFO sqlalchemy.engine.Engine [raw sql] 

In [2]:
service = get_service()

In [6]:
res = service.files().list(
    q=f"mimeType = 'application/vnd.google-apps.folder'",
    fields="nextPageToken, files(id, name, parents)",
    pageSize=10,
).execute()

In [7]:
items = res.get("files", [])

In [8]:
items

[{'id': '1bsi-CJYZiKp1qO6zBfX0DHD2YhXMEloT',
  'name': 'bar',
  'parents': ['1NYVYa3ut0Escl41dVssC9Dd3RYg5yTV0']},
 {'id': '13_KNZLMccgoQrMws2mr02FMf3kg_QpeC',
  'name': 'foo',
  'parents': ['1lgMsEmncNy8UiWa0jItj6z-IfoOYYf8o']},
 {'id': '1NYVYa3ut0Escl41dVssC9Dd3RYg5yTV0',
  'name': 'stock',
  'parents': ['1lgMsEmncNy8UiWa0jItj6z-IfoOYYf8o']},
 {'id': '1lgMsEmncNy8UiWa0jItj6z-IfoOYYf8o', 'name': 'stock'}]